In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import cv2
import time
from datetime import datetime
import pygame

# Initialize Pygame for sound
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound('D:\DSA\project\ohhhh_yeah.mp3')  # Replace 'alarm.wav' with your sound file

initial_state = None
motion_track_list = [None, None]
motion_time = []
data_frame = pd.DataFrame(columns=["Initial", "Final"])

video = cv2.VideoCapture(0)

# Motion sensitivity control (adjustable threshold)
motion_sensitivity = 30  # Initial threshold

# Folder to save frames on motion
frame_save_path = "motion_frames"

while True:
    check, cur_frame = video.read()
    var_motion = 0
    gray_image = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_image, (21, 21), 0)

    if initial_state is None:
        initial_state = gray_frame
        continue

    differ_frame = cv2.absdiff(initial_state, gray_frame)
    thresh_frame = cv2.threshold(differ_frame, motion_sensitivity, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations=2)

    contours, _ = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 10000:
            continue
        var_motion = 1
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(cur_frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        # Save the frame when motion is detected
        if not cv2.imwrite(f"{frame_save_path}/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.jpg", cur_frame):
            print("Error saving frame")

    motion_track_list.append(var_motion)
    motion_track_list = motion_track_list[-2:]

    if motion_track_list[-1] == 1 and motion_track_list[-2] == 0:
        motion_time.append(datetime.now())
        pygame.mixer.Sound.play(alarm_sound)  # Play the alarm sound when motion starts

    if motion_track_list[-1] == 0 and motion_track_list[-2] == 1:
        motion_time.append(datetime.now())

    cv2.imshow("Gray Frame", gray_frame)
    cv2.imshow("Difference Frame", differ_frame)
    cv2.imshow("Threshold Frame", thresh_frame)
    cv2.imshow("Color Frame", cur_frame)

    wait_key = cv2.waitKey(1)

    if wait_key == ord('m'):
        if var_motion == 1:
            motion_time.append(datetime.now())
        break

for a in range(0, len(motion_time), 2):
    data_frame = data_frame.append({"Initial": motion_time[a], "Final": motion_time[a + 1]}, ignore_index=True)

data_frame.to_csv("EachMovement.csv")
video.release()
cv2.destroyAllWindows()


pygame 2.5.1 (SDL 2.28.2, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
